#### Requirements

In [62]:
pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Dataset

In [63]:
import torchaudio
from datasets import load_dataset
from torch.utils.data import Dataset


class GTZAN(Dataset):

    def __init__(self, split="train"):
        self.dataset = load_dataset("marsyas/gtzan", split=split)
        self.labels = ['blues', 'classical', 'country', 'disco', 'hiphop',
                       'jazz', 'metal', 'pop', 'reggae', 'rock']

        self.label2idx = {label: idx for idx, label in enumerate(self.labels)}
        self.n_classes = len(self.labels)

    def __getitem__(self, idx):
        file_path = self.dataset[idx]['file']
        audio, sr = torchaudio.load(file_path)
        label = self.label2idx[self.labels[self.dataset[idx]['genre']]]

        return audio, label

    def __len__(self):
        return len(self.dataset)

In [64]:
import os

def get_dataset(dataset):
    if dataset == "gtzan":
        d = GTZAN()
    else:
        raise NotImplementedError("Dataset not implemented")
    return d

#### Dataset tests

In [65]:
import unittest
import torch

class TestGTZAN(unittest.TestCase):
    def test_dataset(self):
        dataset = GTZAN()
        sample_idx = 0
        sample = dataset.__getitem__(sample_idx)

        # Audio waveform
        self.assertIsInstance(sample[0], torch.Tensor)

        # Label
        self.assertIsInstance(sample[1], int)

        # Audio waveform has at least 1 sample
        self.assertGreaterEqual(sample[0].shape[0], 1)

        # Label is non-negative
        self.assertGreaterEqual(sample[1], 0)
        
        # Label is less than the number of classes in the dataset
        self.assertLess(sample[1], dataset.num_classes)

In [66]:
# gtzan_test = TestGTZAN()
# gtzan_test.test_dataset

### Dataloader

In [67]:
from torch.utils.data import DataLoader
dataset = get_dataset("gtzan")

dataloader = DataLoader(
    dataset,
    batch_size=48,
    num_workers=0,
    drop_last=True,
    shuffle=False,
)

Found cached dataset gtzan (/Users/murphy/.cache/huggingface/datasets/marsyas___gtzan/default/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910)


#### Dataloader tests

In [68]:
class TestDataLoader(unittest.TestCase):
    
    def test_batch_size(self):
        # set up
        dataset = get_dataset("gtzan")
        input_shape = (1, 59049)
        dataloader = DataLoader(dataset, batch_size=48, num_workers=0, drop_last=True, shuffle=False)
        
        # test
        for batch_idx, (data, target) in enumerate(dataloader):
            batch_size = data.shape[0]
            self.assertEqual(batch_size, 48, f"Batch {batch_idx} - Expected batch size: 48, Actual batch size: {batch_size}")

In [69]:
# dataloader_test = TestDataLoader()
# dataloader_test.test_batch_size()

## Model

In [70]:
import torch.nn as nn
import numpy as np


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

    def initialize(self, m):
        if isinstance(m, (nn.Conv1d)):
            # nn.init.xavier_uniform_(m.weight)
            # if m.bias is not None:
            #     nn.init.xavier_uniform_(m.bias)

            nn.init.kaiming_uniform_(m.weight, mode="fan_in", nonlinearity="relu")


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [71]:
class SampleCNN(Model):
    def __init__(self, strides):
        super(SampleCNN, self).__init__()

        self.strides = strides
        self.sequential = [
            nn.Sequential(
                nn.Conv1d(1, 128, kernel_size=3, stride=3, padding=0),
                nn.BatchNorm1d(128),
                nn.ReLU(),
            )
        ]

        self.hidden = [
            [128, 128],
            [128, 128],
            [128, 256],
            [256, 256],
            [256, 256],
            [256, 256],
            [256, 256],
            [256, 256],
            [256, 512],
        ]

        assert len(self.hidden) == len(
            self.strides
        ), "Number of hidden layers and strides are not equal"
        for stride, (h_in, h_out) in zip(self.strides, self.hidden):
            self.sequential.append(
                nn.Sequential(
                    nn.Conv1d(h_in, h_out, kernel_size=stride, stride=1, padding=1),
                    nn.BatchNorm1d(h_out),
                    nn.ReLU(),
                    nn.MaxPool1d(stride, stride=stride),
                )
            )

        # 1 x 512
        self.sequential.append(
            nn.Sequential(
                nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(512),
                nn.ReLU(),
            )
        )

        self.sequential = nn.Sequential(*self.sequential)

    def forward(self, x):
        out = self.sequential(x)
        return x

In [72]:
from collections import OrderedDict


def load_encoder_checkpoint(checkpoint_path: str) -> OrderedDict:
    state_dict = torch.load(checkpoint_path, map_location=torch.device("cpu"))
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        if "encoder." in k:
            new_state_dict[k.replace("encoder.", "")] = v

    return new_state_dict

In [73]:
import pickle
strides = [3, 3, 3, 3, 3, 3, 3, 3, 3]
model = SampleCNN(strides)
checkpoint_path = "../CLMR/clmr_checkpoints/clmr_checkpoint_10000/clmr_checkpoint_10000.pt"
model.load_state_dict(load_encoder_checkpoint(checkpoint_path))
with open('model.pkl', 'wb') as f:
  pickle.dump(model, f)

In [74]:
model.eval()  # put the model in evaluation mode

all_representations = []
with torch.no_grad():
    for data, target in dataloader:
        # pass the data through the model to get representations
        representations = model(data)
        all_representations.append(representations)

all_representations = torch.cat(all_representations)
with open('representations.pkl', 'wb') as f:
  pickle.dump(all_representations, f)

MemoryError: Cannot allocate write+execute memory for ffi.callback(). You might be running on a system that prevents this. For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks